In [1]:
import pandas as pd

In [2]:
df = pd.read_pickle('data/train.pkl')

In [3]:
df.head(2)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,bureau_AMT_CREDIT_MAX_OVERDUE_divide_DAYS_CREDIT_UPDATE_max,bureau_AMT_CREDIT_MAX_OVERDUE_divide_DAYS_CREDIT_UPDATE_min,bureau_AMT_CREDIT_MAX_OVERDUE_divide_DAYS_CREDIT_UPDATE_sum,bureau_AMT_CREDIT_MAX_OVERDUE_divide_DAYS_CREDIT_UPDATE_std,bureau_STATUS_NUM_divide_MONTHS_BALANCE_sum_mean,bureau_STATUS_NUM_divide_MONTHS_BALANCE_sum_median,bureau_STATUS_NUM_divide_MONTHS_BALANCE_sum_max,bureau_STATUS_NUM_divide_MONTHS_BALANCE_sum_min,bureau_STATUS_NUM_divide_MONTHS_BALANCE_sum_sum,bureau_STATUS_NUM_divide_MONTHS_BALANCE_sum_std
0,100002,1.0,0,0,0,0,0,202500.0,406597.5,24700.5,...,-0.0,-152.837727,-158.26821,67.760926,-0.135012,-0.136674,0.0,-0.258263,-1.080096,0.110696
1,100003,0.0,0,1,0,1,0,270000.0,1293502.5,35698.5,...,-0.0,-0.000000,0.00000,0.000000,NaN,NaN,NaN,NaN,0.000000,NaN


In [4]:
null_counts = df.isnull().sum()

In [5]:
null_counts = null_counts[null_counts>0]

In [6]:
print(null_counts[:3])
print('=========')
print(len(null_counts))

AMT_ANNUITY            12
AMT_GOODS_PRICE       278
OWN_CAR_AGE        202929
dtype: int64
2274


In [7]:
null_ratios = null_counts / len(df)

In [8]:
null_ratios[:3]

AMT_ANNUITY        0.000039
AMT_GOODS_PRICE    0.000904
OWN_CAR_AGE        0.659908
dtype: float64

In [9]:
null_thresh = 0.8

In [10]:
null_cols = null_ratios[null_ratios > null_thresh].index

In [11]:
len(null_cols), null_cols[:3]

(219, Index(['DEF_60_CNT_SOCIAL_CIRCLE_divide_DEF_30_CNT_SOCIAL_CIRCLE',
        'AMT_REQ_CREDIT_BUREAU_HOUR_divide_DEF_30_CNT_SOCIAL_CIRCLE',
        'AMT_REQ_CREDIT_BUREAU_HOUR_divide_DEF_60_CNT_SOCIAL_CIRCLE'],
       dtype='object'))

In [12]:
df.drop(null_cols, axis=1, inplace=True)

In [13]:
print('Columns dropped for being over {}% null:'.format(100*null_thresh))

Columns dropped for being over 80.0% null:


In [14]:
categorical_feats = ["FLAG_OWN_CAR","FLAG_OWN_REALTY","NAME_TYPE_SUITE","NAME_INCOME_TYPE","NAME_EDUCATION_TYPE","NAME_FAMILY_STATUS","NAME_HOUSING_TYPE","OCCUPATION_TYPE","FLAG_DOCUMENT_2","FLAG_DOCUMENT_3","FLAG_DOCUMENT_4","FLAG_DOCUMENT_5","FLAG_DOCUMENT_6","FLAG_DOCUMENT_7","FLAG_DOCUMENT_8","FLAG_DOCUMENT_9","FLAG_DOCUMENT_10","FLAG_DOCUMENT_11","FLAG_DOCUMENT_12","FLAG_DOCUMENT_13","FLAG_DOCUMENT_14","FLAG_DOCUMENT_15","FLAG_DOCUMENT_16","FLAG_DOCUMENT_17","FLAG_DOCUMENT_18","FLAG_DOCUMENT_19","FLAG_DOCUMENT_20","FLAG_DOCUMENT_21","NAME_TYPE_SUITE"]

In [15]:
for col in null_cols:
    if col in categorical_feats:
        categorical_feats.pop(col) # since cat feats are label encoded, there won't be any missing value
        print(col) # expect to print nothing

In [16]:
import numpy as np
df.replace([np.inf, -np.inf], np.nan, inplace=True)

In [17]:
df.fillna(-1, inplace=True)

In [18]:
df.shape

(307511, 2176)

In [19]:
numeric_feats = list(set(df.columns.values) - set(categorical_feats) - set(['TARGET']))

In [20]:
len(numeric_feats)

2147

In [21]:
import numpy as np
cat_feats_idx = np.array([df.columns.get_loc(x) for x in categorical_feats])
cat_feat_lookup = pd.DataFrame({'feature': categorical_feats, 'column_index': cat_feats_idx})
cat_feat_lookup.head()

,feature,column_index
0,FLAG_OWN_CAR,4
1,FLAG_OWN_REALTY,5
2,NAME_TYPE_SUITE,11
3,NAME_INCOME_TYPE,12
4,NAME_EDUCATION_TYPE,13


In [22]:
# int_feats_idx = [df.columns.get_loc(x) for x in numeric_feats]
numeric_feats_idx = np.array(
    [df.columns.get_loc(x) 
     for x in numeric_feats]
)
numeric_feat_lookup = pd.DataFrame(
    {'feature': numeric_feats, 
     'column_index': numeric_feats_idx}
)
numeric_feat_lookup.head()

,feature,column_index
0,prev_onehot_NAME_SELLER_INDUSTRY_Connectivity_...,467
1,creditcard_AMT_RECIVABLE_divide_SK_DPD_minus_S...,1439
2,prev_AMT_CREDIT_divide_DAYS_LAST_DUE_1ST_VERSI...,1101
3,prev_DAYS_LAST_DUE_sum,969
4,def_divide_count_prev_sum,1173


In [ ]:
numeric_feat_lookup.shape

In [23]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

## for col in numeric_feats:
    print(col)
    scaler.fit_transform((abs(df2[col])+200).values.reshape(-1,1))

In [24]:
final_col_names = df.columns
# df = df.values

In [34]:
len(final_col_names)

2176

In [26]:
scaler = StandardScaler()
for col in numeric_feats:
    df[col] = scaler.fit_transform(df[col].values.reshape(-1,1))
# df[:, numeric_feats_idx] = scaler.fit_transform(df[:, numeric_feats_idx])

/home/kai/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [ ]:
scaler_2 = MinMaxScaler(feature_range=(0, 1))
for col in categorical_feats:
    df[col] = scaler_2.fit_transform(df[col].values.reshape(-1,1))
# df[:, cat_feats_idx] = scaler_2.fit_transform(df[:, cat_feats_idx])

In [ ]:
import gc
gc.collect()

In [27]:
test = df.tail()

In [28]:
test.to_pickle('data/test_pre.pkl')

In [29]:
df.to_pickle('data/train_pre.pkl')